In [1]:
!pip install pymysql sshtunnel pandarallel eng textblob
!python3 -m eng .
import pandas as pd
import numpy as np
import logging
import matplotlib.pyplot as plt
import seaborn as sns 

import trait_functions as tf

import warnings
warnings.filterwarnings("ignore")

zsh:1: command not found: pip
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/eng/__main__.py", line 3, in <module>
    entrypoint()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/eng/_cli.py", line 66, in entrypoint
    sys.exit(main(sys.argv[1:]))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/eng/_cli.py", line 53, in main
    old_content = path.read_text(encoding=args.encoding)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/pathlib.py", line 1135, in read_text
    return f.read()
  File "/Library/Frameworks

ModuleNotFoundError: No module named 'trait_functions'

In [395]:
#just copying down the preliminary steps for fleshing out the vocabulary

df_slide = pd.read_csv("scripts_preprocessed_slide_nltk_10000.csv", index_col=0)
df = pd.read_csv("scripts_preprocessed_df_spark_10000.csv", index_col=0)

In [396]:
idioms = pd.read_csv("idioms.csv")
bigram_list = pd.read_csv("bigram_list_2.csv")
unigram_list = pd.read_csv("unigram_list_2.csv")

In [397]:
feature_names = set(idioms) | set(bigram_list['0']) | set(unigram_list['0'])

In [398]:
#this maps the top twenty titles for each cluster
#suppose we should also retrieve the bottom twenty titles
entertainment_titles = pd.read_csv("entertainment_titles.csv")

In [399]:
entertainment_titles

,cluster_id,title_1,title_2,title_3,title_4,title_5,title_6,title_7,title_8,title_9,...,title_11,title_12,title_13,title_14,title_15,title_16,title_17,title_18,title_19,title_20
0,1,598,1606,1888,6100,6674,6778,7320,7689.0,14812.0,...,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1093,3419,6100,6461,6758,7771,5210,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,7749,5571,6439,6499,3683,3717,4882,1743.0,6484.0,...,3421.0,4229.0,6068.0,5740.0,6042.0,5175.0,5491.0,867.0,842.0,292.0
3,4,2807,7810,6644,1917,1197,6074,4985,7741.0,3683.0,...,842.0,7811.0,6265.0,1618.0,3419.0,6203.0,2837.0,7391.0,445.0,2771.0
4,5,2807,7810,2837,4852,1197,6662,6644,7811.0,6704.0,...,1693.0,4882.0,1917.0,2971.0,1743.0,2771.0,5211.0,4985.0,6203.0,6265.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,510,7682,4985,1009,6100,6607,2837,5057,3419.0,5211.0,...,24729.0,6644.0,6295.0,6756.0,7391.0,5047.0,1197.0,2275.0,2483.0,1888.0
510,511,6793,4985,598,6439,3421,22,7749,5608.0,6295.0,...,1618.0,6607.0,2837.0,1917.0,6790.0,5057.0,5004.0,422.0,6074.0,3934.0
511,512,445,1917,7810,5047,1197,6062,4229,6125.0,5175.0,...,6295.0,7682.0,6644.0,1743.0,5608.0,4985.0,598.0,5210.0,1693.0,6607.0
512,513,6607,1917,5057,4229,1453,7682,6438,6534.0,7600.0,...,7272.0,842.0,2413.0,4985.0,3934.0,7658.0,4852.0,7391.0,7320.0,7811.0


In [400]:
#we should just take the, say a table of all the movie titles and then, say, the top twenty clusters and their percentage likes 
#first finding how many like
from trait_functions import qry


sql = '''with cte1 as (
    select likable_id title_id, user_id from likeable_likes
    where likable_type = 'title'),
    cte2 as (
    select title_id
    from cte1
    group by 1
    having count(distinct user_id) >=30)
    select title_id, user_id from cte1
    where title_id in (select title_id from cte2)
'''
title_likes = qry(sql)
title_likes["title_id"] = title_likes["title_id"].astype('int64')
title_likes["user_id"] = character_likes["user_id"].astype('int64')
title_usercounts = title_likes.groupby('title_id').count()
title_likes

,title_id,user_id
0,1009,103306
1,1009,1154098
2,1009,1177079
3,1009,118242
4,1009,1474137
...,...,...
550518,9865,603754
550519,9865,6039542
550520,9865,6041658
550521,9865,6045669


In [401]:
user_top_trait_category_df = pd.read_parquet("cleaned_9m_user_traits_cluster_id.parquet")
user_top_trait_category_df
#maps user to cluster
#now we replace the user_id column in title_likes with cluster ids

,top_category,label,title_likes,character_likes,has_liked,cluster_id
user_id,,,,,,
1,medium,0,0.0,0.0,False,190
24,smooth,15,4.0,31.0,True,395
26,happy,29,34.0,35.0,True,119
27,solitary,0,2.0,3.0,True,457
28,sad,33,0.0,3.0,True,310
...,...,...,...,...,...,...
8909388,optimistic,7,0.0,0.0,False,199
8909389,brainy,10,0.0,0.0,False,14
8909390,brainy,13,0.0,0.0,False,17


In [164]:
# new_title_likes = title_likes.head(50)

In [403]:
#Apparently these user ids are present in the title_likes, but not in the user_to_cluster file. Not sure why, removing them for now
not_in_users = [108, 47, 51, 1024, 113, 337, 1537, 387, 1129, 554, 4900, 948, 1123, 310, 76, 466, 408, 485, 201002, 5050, 4282, 844, 448]
mask = [user_id not in not_in_users for user_id in title_likes['user_id']]
title_likes = title_likes[mask]

In [404]:

# new_title_likes['cluster_id'] = new_title_likes['user_id']
title_likes['cluster_id'] = user_top_trait_category_df.loc[title_likes['user_id'], 'cluster_id'].values
# title_likes.loc[title_likes['user_id'] == user,'cluster_id'] = user_top_trait_category_df.at[user, 'cluster_id']
title_likes.head()

,title_id,user_id,cluster_id
0,1009,103306,154
1,1009,1154098,401
2,1009,1177079,107
3,1009,118242,470
4,1009,1474137,13


In [406]:
#now calculating the percentage of each title liked by each cluster
title_cluster_percentage_likes= pd.DataFrame()

title_cluster_percentage_likes['title_id'] = title_likes['title_id'].unique()
#going through each title and fetching the top ten clusters by percentage
#adding all clusters as column values here
for i in range(1,title_likes['cluster_id'].max() + 1):
    title_cluster_percentage_likes[str(i)] = 0
#only keeping the titles that are in the dataframe
#should make this processing quicker
title_cluster_percentage_likes = title_cluster_percentage_likes[title_cluster_percentage_likes.index.isin(df.index)]
for title in title_cluster_percentage_likes['title_id']:
    #so this should be total cluster likes rather than the likes of just that title
    #getting the an array of the total likes of the clusters that have liked the title
    total_likes = title_likes['cluster_id'].value_counts()[title_likes[title_likes['title_id'] == title]['cluster_id'].value_counts().index]
    list_thing = (title_likes[title_likes['title_id'] == title]['cluster_id'].value_counts()/total_likes.tolist())
    list_thing = list_thing.sort_values(ascending=False)
    #only keeping the top hundred percentages, and converting to percentages
    list_thing = list_thing.head(100) * 100
    list_thing
    #need to add all 512 clusters to the dataframe
    for index in list_thing.index:
        title_cluster_percentage_likes.loc[title_cluster_percentage_likes['title_id'] == title,str(index)] = list_thing[index]



In [408]:
title_cluster_percentage_likes.shape


(124, 515)

In [409]:
title_cluster_percentage_likes.to_csv("title_cluster_percentage_likes.csv")


In [410]:
# train test split

In [411]:
def train_test_split_by_title_indices(y, test_size = 0.2, random_state = 1871):
    indices = y.index.unique().to_numpy()
    num_of_test = round(test_size*len(indices))
    np.random.seed(seed=random_state)
    np.random.shuffle(indices)
    test_index, train_index = indices[:num_of_test], indices[num_of_test:]
    #X_train, y_train = X.loc[train_index], y.loc[train_index]
    #X_test, y_test = X.loc[test_index], y.loc[test_index]
    return train_index, test_index

In [412]:
train_index, test_index = train_test_split_by_title_indices(title_cluster_percentage_likes, test_size = 0.2, random_state = 1871)

In [413]:
# some titles appear not to be subtitled--> sorting them out

# #for the train indice
# train_not_in_slide = []
# for index in train_index:
#     if index not in df_slide.index:
#         train_not_in_slide.append(index)
# new_train_index = list(set(train_index) - set(train_not_in_slide))

# #for the test indices
# test_not_in_slide = []
# for index in test_index:
#     if index not in df_slide.index:
#         test_not_in_slide.append(index)
# new_test_index = list(set(test_index) - set(test_not_in_slide))


df_slide_train = df_slide.loc[train_index]
df_slide_test = df_slide.loc[test_index]

df_train = df.loc[train_index]
df_test = df.loc[test_index]

df_slide_train

,subtitles,preprocessed_text
title_id,,
725,"Are you ready, honey? Yes. Guess what, Mom. Wh...",ready honey yes guess find throw moth air turn...
1000,"Hello, stranger. Sorry. Looking for a cigarett...",hello strange sorry look cigarette give thank ...
385,"Director's cut My name is Ash, and I am a slav...",director cut ash slave close figure year ad dr...
645,"Next subject, Kowalski, Leon. Engineer, waste ...",subject engineer waste disposal section new em...
15,We're all familiar with the illegal poaching.....,familiar illegal poach endanger animal wild an...
...,...,...
912,This is a story of an ordinary little boy... ....,ordinary little boy name bucket fast strong cl...
1287,"Hi, everybody. This is your cousin Brucie. Who...",hi everybody cousin whoa summer romance bloom ...
598,"MAN: Get up! Get up! The winner, by total anni...",man winner total annihilation step ring little...


In [414]:
y_labels = title_cluster_percentage_likes.join(df_slide[['subtitles']], how="inner").drop(columns='subtitles')
y_labels.shape
# y_labels

(124, 515)

In [415]:

y_train = y_labels.loc[train_index]
y_test = y_labels.loc[test_index]
print(y_train.shape)
print(y_test.shape)

(99, 515)
(25, 515)


In [416]:
#TF-IDF processing

In [ ]:
#could change this to nltk later 
path = 'scripts_preprocessed_slide_spark_10000.csv' 
df_slide = pd.read_csv(path).set_index('title_id')

df_slide.head()

,subtitles,preprocessed_text
title_id,,
5,"When the boy was born... ...like all Spartans,...",when boy bear like all inspect would small pun...
11,"This is a story of boy meets girl. The boy, To...",boy meet girl boy new jersey grow up believe w...
13,"Hey! I'll be right with you. So, Cameron. Here...",hey right with so here go nine school in year ...
15,We're all familiar with the illegal poaching.....,all familiar with illegal poach endanger anima...
17,Digga-digga-dog Digga-digga-dog - Digga-digga-...,dog dog dog dog dog dog aint man in moon head ...


In [418]:
#tf idf
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(
    vocabulary = feature_names,
    #min_df=0.015,
    #max_features=20000,
    ngram_range=(1, 13),
    use_idf = True,#False, # can try True to use normal TF-IDF -> this removes the IDF aspect
    binary = False
)
#applying tf idf to training data
X_tf_idf = tf_idf.fit_transform(df_slide_train['preprocessed_text'])

print("n_samples: %d, n_features: %d" % X_tf_idf.shape)

n_samples: 99, n_features: 24115


In [419]:
# Getting the feature names (words) from the TfidfVectorizer
feature_names = tf_idf.get_feature_names_out()

# Converting the sparse matrix to a dense matrix
X_tf_dense = X_tf_idf.toarray()

# Creating a Pandas DataFrame using the dense matrix and the feature names as column names
X_tf_df = pd.DataFrame(X_tf_dense, columns=feature_names)

In [420]:
tf_idf_results = df_slide_train.reset_index()
tf_idf_results = tf_idf_results.join(X_tf_df)
tf_idf_results = tf_idf_results.set_index('title_id').drop(columns=['subtitles', 'preprocessed_text'])
tf_idf_results

,aa,abandon,abbey,abduct,abduction,abet,abide,ability,ability to,able,...,yummy,zebra,zeppelin,zero,zip,zipper,zombie,zone,zoo,zoom
title_id,,,,,,,,,,,,,,,,,,,,,
725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.013436,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.074816,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
1287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021163,0.0,0.000000,...,0.0,0.0,0.0,0.093793,0.0,0.0,0.0,0.0,0.000000,0.0


In [421]:
tf_idf_results = tf_idf_results.loc[:,(tf_idf_results!=0).any(axis=0)]
tf_idf_results

,aa,abandon,abduct,abide,ability,able,aboard,abomination,abortion,abroad,...,youth,youve,yuck,yummy,zeppelin,zero,zip,zipper,zone,zoo
title_id,,,,,,,,,,,,,,,,,,,,,
725,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.057524,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.031749,0.085432,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
385,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
645,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.008789,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
15,0.0,0.0,0.0,0.0,0.000000,0.013436,0.0,0.000000,0.0,0.0,...,0.000000,0.037281,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.074816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.005752,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1287,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.008984,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
598,0.0,0.0,0.0,0.0,0.021163,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.016165,0.000000,0.0,0.0,0.093793,0.0,0.0,0.0,0.000000


In [422]:
# re-tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(
    vocabulary = tf_idf_results.columns,
    #min_df=0.015,
    #max_features=20000,
    ngram_range=(1, 13),
    use_idf = True,#False, # can try True to use normal TF-IDF -> this removes the IDF aspect
    binary = False
)

#applying tf idf to training data
X_tf_idf = tf_idf.fit_transform(df_slide_train['preprocessed_text'])

X_tf_idf_test = tf_idf.transform(df_slide_test['preprocessed_text'])

print("n_samples: %d, n_features: %d" % X_tf_idf_test.shape)

n_samples: 25, n_features: 10316


In [423]:
# Getting the feature names (words) from the TfidfVectorizer
feature_names = tf_idf.get_feature_names_out()

# Converting the sparse matrix to a dense matrix
X_tf_dense_test = X_tf_idf_test.toarray()

# Creating a Pandas DataFrame using the dense matrix and the feature names as column names
X_tf_df_test = pd.DataFrame(X_tf_dense_test, columns=feature_names)

tf_idf_test = df_slide_test.reset_index()
tf_idf_test = tf_idf_test.join(X_tf_df_test)
tf_idf_test = tf_idf_test.set_index('title_id').drop(columns=['subtitles', 'preprocessed_text'])

In [424]:
tf_idf_test

,aa,abandon,abduct,abide,ability,able,aboard,abomination,abortion,abroad,...,youth,youve,yuck,yummy,zeppelin,zero,zip,zipper,zone,zoo
title_id,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.022034,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
952,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.036718,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
872,0.0,0.000000,0.0,0.000000,0.000000,0.015830,0.000000,0.0,0.000000,0.0,...,0.0,0.035138,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
292,0.0,0.000000,0.0,0.000000,0.000000,0.015633,0.000000,0.0,0.000000,0.0,...,0.0,0.034701,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
772,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.007579,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
387,0.0,0.000000,0.0,0.000000,0.000000,0.047749,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.025624,0.000000,0.0,0.0,0.000000
380,0.0,0.020584,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.028678,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
547,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.008592,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
1253,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.007740,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000


In [425]:
#feature selection

In [426]:
from scipy.stats import ttest_ind
def select_datasets(tf_idf_data, per, trait, high=80, low=20):
    use_titles_80 = per.loc[per[trait] >= high].index
    use_titles_20 = per.loc[per[trait] <= low].index
    sort_data = tf_idf_data.sort_index()
    top_80 = sort_data.loc[:, use_titles_80]
    bot_20 = sort_data.loc[:, use_titles_20]
    return top_80, bot_20

def calculate_basic_statistics(top_80, bot_20):
    words = top_80.index
    final_res = pd.DataFrame(index=top_80.index)
    final_res["top_mean"] = top_80.mean(axis=1)
    final_res["bot_mean"] = bot_20.mean(axis=1)
    final_res["top_var"] = top_80.var(axis=1)
    final_res["bot_var"] = bot_20.var(axis=1)
    final_res["top_n"] = top_80.shape[1]
    final_res["bot_n"] = bot_20.shape[1]
    final_res["top_mean/bot_mean"] = final_res["top_mean"]/final_res["bot_mean"]
    return final_res

def select_feature_words_robust(tf_idf_data, per, trait, high, low, alternative, p_threshold, top_bot_ratio_threshold=1.01):
    top, bot = select_datasets(tf_idf_data, per, trait, high, low)
    df = calculate_basic_statistics(top, bot)
    df['tstat'], df['pval'] = ttest_ind(top, bot, axis = 1, equal_var = False, alternative = alternative)
    df = df.dropna().loc[(df.pval<p_threshold)&(df.bot_mean>0)&(df.top_mean>0)]
    if alternative=="greater":
        df = df.loc[df["top_mean/bot_mean"]>top_bot_ratio_threshold].sort_values(by="top_mean/bot_mean", ascending=False)
    elif alternative=="less":
        df = df.loc[df["top_mean/bot_mean"]<(1.0/top_bot_ratio_threshold)].sort_values(by="top_mean/bot_mean", ascending=True)
    else:
        df = df.sort_values(by="top_mean/bot_mean", ascending=True)
    return df

In [427]:
tf_idf_res_wo_slide = tf_idf.fit_transform(df_train['preprocessed_text'])
tf_idf_res_wo_slide = pd.DataFrame(tf_idf_res_wo_slide.toarray(), 
                            columns=tf_idf.get_feature_names_out(),
                           index = df_train.index)
tf_idf_res = tf_idf_res_wo_slide.transpose() # the dataset for t-test
tf_idf_res.head()

title_id,725,1000,385,645,15,736,1070,1115,1004,756,...,223,1176,843,1094,730,912,1287,598,524,358
aa,0.0,0.0,0.0,0.0,0.0,0.021588,0.0,0.0,0.0,0.023618,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0
abandon,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00566,0.000000,0.0,0.0,0.000000,0.0,0.0
abduct,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0
abide,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0
ability,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00000,0.007392,0.0,0.0,0.009407,0.0,0.0


In [428]:
tf_idf_res

title_id,725,1000,385,645,15,736,1070,1115,1004,756,...,223,1176,843,1094,730,912,1287,598,524,358
aa,0.000000,0.0,0.0,0.0,0.000000,0.021588,0.00000,0.0,0.0,0.023618,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0
abandon,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.00566,0.000000,0.0,0.0,0.000000,0.0,0.0
abduct,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0
abide,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0
ability,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.00000,0.007392,0.0,0.0,0.009407,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zero,0.005296,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,...,0.000000,0.013145,0.0,0.00000,0.000000,0.0,0.0,0.038524,0.0,0.0
zip,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.01765,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0
zipper,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0
zone,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0


In [429]:
title_per = title_cluster_percentage_likes.sort_index().loc[train_index].rank(pct=True)
title_per

,title_id,1,2,3,4,5,6,7,8,9,...,505,506,507,508,509,510,511,512,513,514
725,0.606061,0.5,0.5,0.464646,0.398990,0.368687,0.378788,0.388889,0.414141,0.338384,...,0.454545,0.914141,0.934343,0.383838,0.454545,0.904040,0.449495,0.429293,0.873737,0.454545
1000,0.616162,0.5,0.5,0.464646,0.979798,0.969697,0.378788,0.388889,0.979798,0.338384,...,0.454545,0.964646,0.934343,0.833333,0.944444,0.439394,0.449495,0.429293,0.873737,0.454545
385,0.626263,0.5,0.5,0.464646,0.828283,0.368687,0.808081,0.843434,0.414141,0.338384,...,0.919192,0.424242,0.454545,0.383838,0.454545,0.439394,0.449495,0.429293,0.414141,0.454545
645,0.636364,0.5,0.5,0.464646,0.398990,0.787879,0.378788,0.388889,0.414141,0.722222,...,0.454545,0.424242,0.454545,0.833333,0.454545,0.439394,0.449495,0.429293,0.873737,0.454545
15,0.646465,0.5,0.5,0.464646,0.398990,0.368687,0.378788,0.388889,0.414141,0.934343,...,0.954545,0.424242,0.454545,0.383838,0.454545,0.439394,0.449495,0.429293,0.414141,0.454545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,0.979798,0.5,0.5,0.464646,0.398990,0.368687,0.378788,0.388889,0.414141,0.803030,...,0.454545,0.424242,0.454545,0.383838,0.454545,0.439394,0.449495,0.429293,0.873737,0.454545
1287,0.989899,0.5,0.5,0.464646,0.398990,0.368687,0.883838,0.388889,0.414141,0.722222,...,0.454545,0.424242,0.454545,0.383838,0.454545,0.439394,0.449495,0.888889,0.414141,0.454545
598,1.000000,0.5,0.5,0.464646,0.398990,0.787879,0.808081,0.388889,0.414141,0.863636,...,0.454545,0.424242,0.454545,0.833333,0.454545,0.439394,0.449495,0.429293,0.414141,0.454545
524,0.595960,0.5,0.5,0.464646,0.398990,0.368687,0.378788,0.388889,0.414141,0.338384,...,0.454545,0.424242,0.454545,0.383838,0.944444,0.439394,0.449495,0.429293,0.414141,0.454545


In [430]:
traits46 = list(title_cluster_percentage_likes.columns)
RANDOM_STATE = 1871

In [431]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_validate
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

import joblib

In [432]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [433]:
from trait_functions import *